# **Using indicator constraints**

***indicators.ipynb***

This example constructs a problem from scratch with variables of various types, adds indicator constraints and shows how to retrieve data regarding indicator coinstraints once they have been added to the problem using the API functions.

&copy; Copyright 2025 Fair Isaac Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.
 
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This example uses FICO&reg; Xpress software. By running it, you agree to the Community License terms of the [Xpress Shrinkwrap License Agreement](https://community.fico.com/s/contentdocument/06980000002h0i5AAA) with respect to the FICO&reg; Xpress software. See the [licensing options](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) overview for additional details and information about obtaining a paid license.

In [ ]:
# Install the xpress package
%pip install -q xpress

Start by creating a problem and adding all variables used in this example.

In [2]:
import xpress as xp

p = xp.problem()

p.controls.miprelstop = 0

# All variables used in this example
v1 = p.addVariable(lb=0, ub=10, vartype=xp.continuous)
v2 = p.addVariable(lb=1, ub=7, vartype=xp.continuous)
v3 = p.addVariable(lb=5, ub=10, threshold=7, vartype=xp.semicontinuous)
vb = p.addVariable(vartype=xp.binary)

Indicator constraints consist of **a tuple** with a condition on a binary variable and a constraint. If the condition on the binary variable holds, then the constraint must apply. The conditional constraints can be linear or nonlinear, equalities or inequalities.

The code cell below creates two objects, each containing a tuple for creating an indicator constraint.

In [ ]:
ind1 = (vb == 0, v1 + v2 >= 6)   # if variable 'vb' is equal to 0, constraint 'v1 + v2 >= 6' applies

ind2 = (vb == 1, v1 + v3 >= 4)   # if variable 'vb' is equal to 1, constraint 'v1 + v3 >= 4' applies

Indicator constraints can conveniently be added by using the [problep.addIndicator()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addIndicator.html) method (note the "==" as the symbol for the equality on the indicator).

The method accepts objects that have been created previously as well as new tuples stated directly as arguments.

In [4]:
p.addIndicator(ind1)                            # adds the first indicator constraint

p.addIndicator((vb == 1, v2 + v3 >= 5), ind2)   # adds another indicator constraint and the second one defined above

The method [problem.getindicators](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.getindicators.html) can be used to get the indicator constraint conditions (indicator variables and complement flags) associated to the rows in a given range.

The complement flags can be:
  - 0: not an indicator constraint
  - 1: for indicator constraints with condition " bin = 1"
  - -1: for indicator constraints with condition " bin = 0"

As we can observe in the output log, all three indicator conditions have the column *C4* as the indicator variable. The first constraint applies when *vb == 0* while the last two constraints are to be applied if *vb == 1*.

In [ ]:
ind_inds = []
ind_flags = []

p.getindicators(ind_inds, ind_flags, 0, 2)

print("Indicator variables and flags: ", ind_inds, ind_flags)

Lastly, we define the objective function as the sum of the variables *v1,v2,v3* and optimize the problem, before printing the solution values for *v1* and *v2*, as well as the whole solution vector and the final objective value.

In [ ]:
p.setObjective(xp.Sum(v1,v2,v3))

p.optimize()

print("v1: ", p.getSolution(v1),
      ", v2: ", p.getSolution(v2),
      "; sol vector: ", p.getSolution(),
      "; obj: ", p.attributes.objval,
      sep="")  # default separator between strings is " "